In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import os
folder_path = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//'
os.chdir(folder_path)


In [ ]:
import os
folder_path = '/content/drive/My Drive/SuperResolution/SuperResolution/Data'
os.chdir(folder_path)
#!wget -O /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR.zip http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip


In [ ]:
!unzip /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR.zip -d /content/drive/MyDrive/SuperResolution/SuperResolution/Data/

Archive:  /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR.zip
   creating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0103.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0413.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0031.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0660.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0126.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0793.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0764.png  
  inflating: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_HR/0550.png  
  inflating: /content/drive/My

In [ ]:
!ls
!cd DIPNet


AIM-track.log  Data  DIPNet  ESRGAN  model.ipynb  README.md  RFDN


In [ ]:
!bash run.sh

Namespace(data_dir='/content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_LR_x8/DIV2K_train_LR_x8', save_dir='/content/drive/MyDrive/SuperResolution/SuperResolution/Data/', model_id=0, include_test=False, hybrid_test=False, ssim=False)
LogHandlers setup!
24-06-17 06:51:37.825 : 00_DIPNet
path: /content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_LR_x8/DIV2K_train_LR_x8/DIV2K_valid_LR/0801x4.png
[ WARN:0@10.452] global loadsave.cpp:248 findDecoder imread_('/content/drive/MyDrive/SuperResolution/SuperResolution/Data/DIV2K_train_LR_x8/DIV2K_train_LR_x8/DIV2K_valid_LR/0801x4.png'): can't open/read file: check file path/integrity
Traceback (most recent call last):
  File "/content/drive/.shortcut-targets-by-id/1-2Aaa2WYrXiEDFEqnYheXGH9k7RzEXXR/SuperResolution/SuperResolution/DIPNet/test_demo.py", line 291, in <module>
    main(args)
  File "/content/drive/.shortcut-targets-by-id/1-2Aaa2WYrXiEDFEqnYheXGH9k7RzEXXR/SuperResolution/SuperResolution/DIPNet/tes

In [2]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [ ]:
import os

# Define the Google Drive directory where you want to save the file
gdrive_dir = '/content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/'

# Ensure the directory exists
os.makedirs(gdrive_dir, exist_ok=True)

# URL of the file to download
url = 'https://data.vision.ee.ethz.ch/yawli/train_x4.tar.gz'

# Use wget to download the file to the specified directory
!wget -P {gdrive_dir} {url}


--2024-06-17 11:04:43--  https://data.vision.ee.ethz.ch/yawli/train_x4.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... ^C


In [ ]:
import tarfile

# Path to the downloaded tar.gz file
tar_path = os.path.join(gdrive_dir, 'train_x4.tar.gz')

# Extract the tar.gz file
with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=gdrive_dir)

print(f'Extracted files to {gdrive_dir}')


Extracted files to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/


In [5]:
import torch
import torch.nn as nn
import RRDBNet_arch as arch


# Assuming your pre-trained model is a .pth file

teacher_model_path = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//models//RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
device = torch.device('cpu')

teacher_model = arch.RRDBNet(3, 3, 64, 23, gc=32)
teacher_model.load_state_dict(torch.load(teacher_model_path), strict=True)



<All keys matched successfully>

In [7]:
import torch.nn as nn
import torch

class StudentModel(nn.Module):
    def __init__(self, scale_factor=4):
        super(StudentModel, self).__init__()
        # Define the upsampling layer first
        self.upsample = nn.Sequential(
            nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(scale_factor)
        )
        #subpixel convolution
        #up convolution

        # Define convolutional layers after upsampling
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = self.upsample(x)  # Upsample first
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.conv3(x)  # Final convolution to refine the image
        return x

# Instantiate the student model
student_model = StudentModel(scale_factor=4)



Feature based noise reduction
Supervised loss on features maps training

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-4)


In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class SuperResolutionDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_size=(64, 64)):
        self.root_dir = root_dir
        self.lr_images = []
        for subdir, _, files in os.walk(root_dir):
            for file in files:
                if file.endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')) and not file.startswith('.'):
                    self.lr_images.append(os.path.join(subdir, file))
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        lr_image_path = self.lr_images[idx]
        lr_image = Image.open(lr_image_path).convert('RGB')

        # Resize image to target size
        lr_image = lr_image.resize(self.target_size, Image.BICUBIC)

        if self.transform:
            lr_image = self.transform(lr_image)

        return lr_image

# Define paths and hyperparameters
lr_dir = '/content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/train'

# Create dataset and dataloader with updated class
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = SuperResolutionDataset(root_dir=lr_dir, transform=transform, target_size=(64, 64))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=12)


In [ ]:
import torch
import torch.optim as optim  # Make sure to import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

def train_student_model(teacher_model, student_model, dataloader, criterion, optimizer, num_epochs=10, save_path='/content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth'):
    teacher_model.eval()  # Teacher model is fixed
    student_model.train() # Student model is in training mode

    if torch.cuda.is_available():
        teacher_model.cuda()
        student_model.cuda()

    for epoch in range(num_epochs):
        for lr_images in dataloader:
            if torch.cuda.is_available():
                lr_images = lr_images.cuda()

            # Forward pass through teacher model
            with torch.no_grad():
                teacher_outputs = teacher_model(lr_images)

            # Forward pass through student model
            student_outputs = student_model(lr_images)

            # Resize teacher's outputs to match the size of student's outputs
            teacher_outputs_resized = torch.nn.functional.interpolate(teacher_outputs, size=student_outputs.shape[2:], mode='bicubic', align_corners=False)

            # Compute loss between teacher's outputs and student's outputs
            loss = criterion(student_outputs, teacher_outputs_resized)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print loss for the epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Save the student model after each epoch
        torch.save(student_model.state_dict(), save_path)
        print(f'Saved model after epoch {epoch+1} to {save_path}')

# Define paths and hyperparameters
save_path = '/content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth'
num_epochs = 10
batch_size = 16

criterion = nn.MSELoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.0008)

# Train the student model
train_student_model(teacher_model, student_model, dataloader, criterion, optimizer, num_epochs=num_epochs, save_path=save_path)

Epoch [1/10], Loss: 0.0092
Saved model after epoch 1 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [2/10], Loss: 0.0073
Saved model after epoch 2 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [3/10], Loss: 0.0051
Saved model after epoch 3 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [4/10], Loss: 0.0062
Saved model after epoch 4 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [5/10], Loss: 0.0065
Saved model after epoch 5 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [6/10], Loss: 0.0073
Saved model after epoch 6 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [7/10], Loss: 0.0059
Saved model after epoch 7 to /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/models/Student_v6.pth
Epoch [8/10], Loss: 

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os

# Define paths
lr_dir = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//LR'
results_dir = '/G://My Drive//SuperResolution//SuperResolution//ESRGAN//results'
model_path = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//models/Student_v6.pth'

# Ensure results directory exists
os.makedirs(results_dir, exist_ok=True)

# Define transformation for input images
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load the trained student model
student_model = StudentModel(scale_factor=4)  # Replace with your student model class
student_model.load_state_dict(torch.load(model_path))
student_model.eval()

if torch.cuda.is_available():
    student_model.cuda()

# Function to upscale and save images
def upscale_images(lr_dir, results_dir, model, transform):
    lr_images = [f for f in sorted(os.listdir(lr_dir)) if os.path.isfile(os.path.join(lr_dir, f)) and not f.startswith('.')]

    for lr_image_name in lr_images:
        lr_image_path = os.path.join(lr_dir, lr_image_name)
        lr_image = Image.open(lr_image_path)

        # Apply transformation
        lr_image_tensor = transform(lr_image).unsqueeze(0)  # Add batch dimension

        if torch.cuda.is_available():
            lr_image_tensor = lr_image_tensor.cuda()

        with torch.no_grad():
            sr_image_tensor = model(lr_image_tensor)

        # Convert tensor back to image
        sr_image = sr_image_tensor.squeeze(0).cpu()
        sr_image = transforms.ToPILImage()(sr_image)
        base_name, ext = os.path.splitext(lr_image_name)
        sr_image_name = f"{base_name}_v6{ext}"
        # Save the upscaled image
        sr_image.save(os.path.join(results_dir, sr_image_name))
        print(f'Saved upscaled image: {os.path.join(results_dir, sr_image_name)}')

# Upscale and save images
upscale_images(lr_dir, results_dir, student_model, transform)


Saved upscaled image: /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/results/Stopsign_v6.jpg
Saved upscaled image: /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/results/baboon_v6.png
Saved upscaled image: /content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/results/comic_v6.png


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

student_params = count_parameters(student_model)
teacher_params = count_parameters(teacher_model)
print(f'The student model has {student_params} trainable parameters.')
print(f'The teacher model has {teacher_params} trainable parameters.')

The student model has 631299 trainable parameters.
The teacher model has 16697987 trainable parameters.


In [ ]:
!pip install torchsummary thop


In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import time
import numpy as np
from thop import profile
from torchsummary import summary
import psutil
from skimage.metrics import structural_similarity as ssim
import cv2
import torchvision.transforms.functional as F

process = psutil.Process(os.getpid())

# Pre-filtering
def prefilter_image(image):
    return cv2.GaussianBlur(image, (3, 3), 0)

# Enhance interpolation
def enhance_interpolation(image, new_size):
    return image.resize(new_size, Image.LANCZOS)

# Post-filtering
def postfilter_image(image):
    return cv2.GaussianBlur(image, (3, 3), 0)

# Denoising
def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 4, 3, 7, 21)

# Median filtering
def median_filter_image(image):
    return cv2.medianBlur(image, 3)

# Sharpening
def sharpen_image(image):
    kernel = np.array([[0, -1, 0], 
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Define your dataset for testing
class TestDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None):
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.lr_images = sorted([f for f in os.listdir(lr_dir) if os.path.isfile(os.path.join(lr_dir, f)) and not f.startswith('.')])
        self.hr_images = sorted([f for f in os.listdir(hr_dir) if os.path.isfile(os.path.join(hr_dir, f)) and not f.startswith('.')])
        self.transform = transform

        print(f"Found {len(self.lr_images)} low-resolution images and {len(self.hr_images)} high-resolution images")

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        lr_image_path = os.path.join(self.lr_dir, self.lr_images[idx])
        hr_image_path = os.path.join(self.hr_dir, self.hr_images[idx])

        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')

        if self.transform:
            lr_image = self.transform(lr_image)
            hr_image = self.transform(hr_image)

        return lr_image, hr_image

# PSNR calculation
def calculate_psnr(img1, img2):
    mse = nn.MSELoss()(img2, img1)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

# SSIM calculation
def calculate_ssim(img1, img2):
    img1_np = img1.squeeze().permute(1, 2, 0).cpu().numpy()
    img2_np = img2.squeeze().permute(1, 2, 0).cpu().numpy()
    return ssim(img1_np, img2_np, data_range=img2_np.max() - img2_np.min(), channel_axis=2)

# Evaluate the model
def evaluate_model(model, dataloader, device, num_images_to_eval=2):
    model.to(device)
    model.eval()

    psnr_list = []
    ssim_list = []
    inference_times = []
    max_memory = 0

    with torch.no_grad():
        print("Starting evaluation...")
        for i, (lr_images, hr_images) in enumerate(dataloader):
            if i >= num_images_to_eval:
                break

            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)

            try:
                # Measure inference time
                start_time = time.time()
                outputs = model(lr_images)
                inference_time = (time.time() - start_time) * 1000  # convert to milliseconds
                inference_times.append(inference_time)

                # Resize outputs to match hr_images dimensions
                outputs_resized = nn.functional.interpolate(outputs, size=hr_images.shape[-2:], mode='bicubic', align_corners=False)
                
                # Convert tensor to numpy for post-processing
                outputs_np = outputs_resized.squeeze().permute(1, 2, 0).cpu().numpy()
                outputs_np = (outputs_np * 255).astype(np.uint8)
                outputs_np = cv2.cvtColor(outputs_np, cv2.COLOR_RGB2BGR)
                
                # Apply filters
                postfiltered_image = postfilter_image(outputs_np)
                denoised_image = denoise_image(postfiltered_image)
                sharpened_image = sharpen_image(denoised_image)
                
                # Convert back to tensor for PSNR and SSIM calculation
                processed_image = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2RGB)
                processed_image = (processed_image / 255.0).astype(np.float32)
                processed_image = torch.tensor(processed_image).permute(2, 0, 1).unsqueeze(0).to(device)
                
                # Calculate PSNR
                psnr = calculate_psnr(processed_image, hr_images)
                psnr_list.append(psnr.item())

                # Calculate SSIM
                ssim_index = calculate_ssim(hr_images, processed_image)
                ssim_list.append(ssim_index)

                # Measure GPU memory usage
                if torch.cuda.is_available():
                    max_memory = max(max_memory, torch.cuda.max_memory_allocated(device) / 1024 ** 2)  # in MB
                else:
                    max_memory = process.memory_info().rss / 1024 ** 2

            except Exception as e:
                print(f"Error during evaluation: {e}")

    avg_psnr = np.mean(psnr_list) if psnr_list else float('nan')
    avg_inference_time = np.mean(inference_times) if inference_times else float('nan')
    avg_ssim = np.mean(ssim_list) if ssim_list else float('nan')
    return avg_inference_time, avg_psnr, max_memory, avg_ssim

# Model summary and FLOPs calculation
def model_profile(model, input_size):
    model.to('cpu')
    summary(model, input_size)
    dummy_input = torch.randn(1, *input_size).to('cpu')
    flops, params = profile(model, inputs=(dummy_input,))
    return flops, params

# Paths
lr_dir = 'G://My Drive//SuperResolution//SuperResolution//Data//DIV2K_train_LR_x8//DIV2K_train_LR_x8'
hr_dir = 'G://My Drive//SuperResolution//SuperResolution//Data//DIV2K_train_HR'
model_path = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//models//SupervisedLossOnFeatureMaps_v2.pth'
teacher_model_path = 'G://My Drive//SuperResolution//SuperResolution//ESRGAN//models//RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth

# Load models
class StudentModel(nn.Module):
    def __init__(self, scale_factor=4):
        super(StudentModel, self).__init__()
        self.upsample = nn.Sequential(
            nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(scale_factor)
        )
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.relu(x)
        return x

student_model = StudentModel(scale_factor=4)
student_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
student_model.eval()

# teacher_model.eval()

# Dataset and DataLoader
transform = transforms.Compose([transforms.ToTensor()])
test_dataset = TestDataset(lr_dir=lr_dir, hr_dir=hr_dir, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Evaluate the models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

avg_inference_time_student, avg_psnr_student, max_memory_student, student_ssim = evaluate_model(student_model, test_dataloader, device)
# avg_inference_time_teacher, avg_psnr_teacher, max_memory_teacher, teacher_ssim = evaluate_model(teacher_model, test_dataloader, device)

# Model profile
input_size = (3, 64, 64)
flops_student, params_student = model_profile(student_model, input_size)
# flops_teacher, params_teacher = model_profile(teacher_model, input_size)

# Print results for student model
print(f'Student Model Evaluation:')
print(f'Average Inference Time: {avg_inference_time_student:.2f} ms')
print(f'Average PSNR: {avg_psnr_student:.2f} dB')
print(f'Average SSIM: {student_ssim}')
print(f'Number of Parameters: {params_student:,}')
print(f'FLOPs: {flops_student:,}')
print(f'GPU Memory Usage: {max_memory_student:.2f} MB')

# # Print results for teacher model
# print(f'\nTeacher Model Evaluation:')
# print(f'Average Inference Time: {avg_inference_time_teacher:.2f} ms')
# print(f'Average PSNR: {avg_psnr_teacher:.2f} dB')
# print(f'Average SSIM: {teacher_ssim}')
# print(f'Number of Parameters: {params_teacher:,}')
# print(f'FLOPs: {flops_teacher:,}')
# print(f'GPU Memory Usage: {max_memory_teacher:.2f} MB')


Found 801 low-resolution images and 801 high-resolution images
Starting evaluation...
Error during evaluation: 'Tensor' object has no attribute 'astype'


KeyboardInterrupt: 